### viewers setup

In [1]:
from IPython.display import display
from notebook_viewer import NotebookViewer
notebook = NotebookViewer()

### loaders initialization

In [2]:
from brainbuilder.utils import genbrain as gb
from brainbuilder.utils import bbp
from brainbuilder.utils import traits as tt

cells = gb.CellCollection()

# Brain Builder modules

##   Build.Region: Region of Interest

In [3]:
from brainbuilder.select_region import select_region, select_hemisphere

### input parameters

In [4]:
annotation = gb.MetaIO.load("../data/P56_Mouse_annotation/annotation.mhd")
hierarchy = gb.load_hierarchy("../data/P56_Mouse_annotation/annotation_hierarchy.json")['msg'][0]
full_density = gb.MetaIO.load("../data/atlasVolume/atlasVolume.mhd")
region_name = "Primary somatosensory area"

### run module

In [5]:
density_raw = select_region(annotation.raw, full_density.raw, hierarchy, region_name)
density_raw = select_hemisphere(density_raw)

### output

In [6]:
notebook.show_metaio('density', full_density.mhd, density_raw)

##  Build.Cells:  Cell Positions

In [7]:
from brainbuilder.cell_positioning import cell_positioning

### input parameters

In [8]:
voxel_dimensions = full_density.mhd['ElementSpacing']
total_cell_count = 200000

### run module

In [9]:
cells.positions = cell_positioning(density_raw, voxel_dimensions, total_cell_count)

### output

In [10]:
notebook.show_points('positions', cells.positions)

## Build.EI:  E-I ratios

In [11]:
from brainbuilder.assignment_synapse_class import assign_synapse_class_from_spatial_dist

### input parameters

In [12]:
recipe_filename = "../data/bbp_recipe/builderRecipeAllPathways.xml"
recipe_sdist = bbp.load_recipe_as_spatial_distribution(recipe_filename,annotation.raw, hierarchy, region_name)
sclass_sdist = tt.reduce_distribution_collection(recipe_sdist, 'synapse_class')

### run module

In [13]:
chosen_synapse_class = assign_synapse_class_from_spatial_dist(cells.positions, sclass_sdist, voxel_dimensions)
cells.add_properties(chosen_synapse_class)

### output

In [14]:
notebook.show_property('synapse_class', cells.positions, cells.properties.synapse_class)

## Build.Composition.ME: METype for Soma

In [15]:
from brainbuilder.assignment_metype import assign_metype

### run module

In [16]:
chosen_me = assign_metype(cells.positions, cells.properties.synapse_class, recipe_sdist, voxel_dimensions)
cells.add_properties(chosen_me)

### output

#### mtypes

In [17]:
notebook.show_property('mtype', cells.positions, cells.properties.mtype)

#### etypes

In [18]:
notebook.show_property('etype', cells.positions, cells.properties.etype)

## Build.Placement: Morphology assignment

In [19]:
from brainbuilder.assignment_morphology import assign_morphology

### input parameters

In [20]:
neurondb_filename = "../data/bbp_recipe/neurondb.dat"
neuron_sdist = bbp.load_neurondb_v4_as_spatial_distribution(neurondb_filename, annotation.raw,hierarchy, region_name,percentile=0.92)

### run module

In [21]:
chosen_morphology = assign_morphology(cells.positions, cells.properties[['mtype', 'etype']], neuron_sdist, voxel_dimensions)
cells.add_properties(chosen_morphology)

### output

In [22]:
notebook.show_property('morphology', cells.positions, cells.properties.morphology)

## Orientation assignement

In [23]:
from brainbuilder.orientation_fields import compute_sscx_orientation_fields
from brainbuilder.assignment_orientation import assign_orientations

### run module

In [24]:
orientation_field = compute_sscx_orientation_fields(annotation, hierarchy, region_name)
cells.orientations = assign_orientations(cells.positions, orientation_field, voxel_dimensions)

### output

In [25]:
notebook.show_placement('placement', cells.positions, cells.orientations, cells.properties.morphology)